In [55]:
!unzip abc.zip -d /content/

In [56]:
%cd /content/

In [57]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16

def load_images_labels(images_folder):
    images = []
    labels = []
    
    for filename in os.listdir(images_folder):
        if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith('.png'):
            image = cv2.imread(os.path.join(images_folder, filename))
            images.append(image)
            label = filename.split('.')[0]
            labels.append(label)
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Load the images and labels from the folders
train_images, train_labels = load_images_labels('/content/Train')
test_images, test_labels = load_images_labels('/content/Test')

# Preprocessing the images to be ready for the model
train_images = np.array([cv2.resize(image, (224, 224)) for image in train_images])
test_images = np.array([cv2.resize(image, (224, 224)) for image in test_images])

# One hot encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

onehot_encoder = OneHotEncoder()
train_labels = onehot_encoder.fit_transform(train_labels.reshape(-1, 1)).toarray()
test_labels = onehot_encoder.transform(test_labels.reshape(-1, 1)).toarray()

# Load the pre-trained model and freeze its layers
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
for layer in vgg.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = vgg.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=predictions)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x=train_images, y=train_labels, batch_size=32, epochs=50, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

<ipython-input-57-692d0f4d61ff>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 28, 28, 75)        750       
                                                                 
 batch_normalization_51 (Bat  (None, 28, 28, 75)       300       
 chNormalization)                                                
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 14, 14, 75)       0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 14, 14, 50)        33800     
                                                                 
 batch_normalization_52 (Bat  (None, 14, 14, 50)       200       
 chNormalization)                                                
                                                     